## Extraction of Board Square Coordinates

<font size="4"> 
<br>
Purpose of Notebook : extraction of squares into a CSV file ( Dynamically )
    <br><br>
</font>

## Necessary Libraries

In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import depthai as dai
import csv
import time


<br><br><br>

## Functional Approach : All the image processing steps that required for Extracting Squares coordinates

In [2]:
def gaussian_blur(image):
    gray_image=cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    gaussian_blur = cv2.GaussianBlur(gray_image,(5,5),0)
    
    return gaussian_blur

def otsu_threshold(gaussian_blur):
    ret2,otsu_binary = cv2.threshold(gaussian_blur,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    return otsu_binary

def canny_edge_detection(otsu_binary):
    global canny
    
    canny = cv2.Canny(otsu_binary,20,255)
    return canny

def dilation1(canny):
    kernel = np.ones((7, 7), np.uint8) 
    img_dilation = cv2.dilate(canny, kernel, iterations=1) 

    return img_dilation

def hough_lines(img_dilation):
    global lines , board_contours
    
     # This returns an array of r and theta values
    lines = cv2.HoughLinesP(img_dilation, 1, np.pi/180, threshold=200, minLineLength=100, maxLineGap=50)
    
    if lines is not None:
        for i, line in enumerate(lines):
            x1, y1, x2, y2 = line[0]
            dy = (y2 - y1)
            dx = (x2 - x1)
            # convert radian to degree and extract angle
            angle = np.rad2deg(np.arctan2(dy, dx))
            
            # Since the Y-axis increases downwards(in opencv), invert the angle.
            angle = 180 - angle if angle > 0 else -angle
            
            # different color for every line
            color = (255,255,255)
            
            cv2.line(img_dilation, (x1, y1), (x2, y2), color, 2)
            
    kernel = np.ones((3, 3), np.uint8) 
  
    img_dilation = cv2.dilate(img_dilation, kernel, iterations=1) 

    board_contours, hierarchy = cv2.findContours(img_dilation, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    return board_contours

def find_contours(board_contours):
    square_centers=list()

    # Copy the board image
    board_squared = canny.copy() # CANNY IMAGE 
    
    for contour in board_contours:
        if 3500 < cv2.contourArea(contour) < 20000:
            # Approximate the contour to a simpler shape
            epsilon = 0.02 * cv2.arcLength(contour, True)
            approx = cv2.approxPolyDP(contour, epsilon, True)
    
    
            
            
            # Ensure the approximated contour has 4 points (quadrilateral)
            if len(approx) == 4:
                pts = [pt[0] for pt in approx]  # Extract coordinates
    
                # Define the points explicitly
                pt1 = tuple(pts[0])
                pt2 = tuple(pts[1])
                pt4 = tuple(pts[2])
                pt3 = tuple(pts[3])
    
                x, y, w, h = cv2.boundingRect(contour)
                center_x=(y+(y+h))/2
                center_y=(x+(x+w))/2
    
                square_centers.append([center_x,center_y,pt2,pt1,pt3,pt4])
    
                 
    
                # Draw the lines between the points
                cv2.line(board_squared, pt1, pt2, (255, 255, 0), 7)
                cv2.line(board_squared, pt1, pt3, (255, 255, 0), 7)
                cv2.line(board_squared, pt2, pt4, (255, 255, 0), 7)
                cv2.line(board_squared, pt3, pt4, (255, 255, 0), 7)


    return board_squared , square_centers

def sort_coordinates(square_centers):

    sorted_coordinates = sorted(square_centers, key=lambda x: x[0], reverse=True)

    if len(sorted_coordinates)>2:

        # Step 1: Group the coordinates by rows where the difference is less than 50
        groups = []
        current_group = [sorted_coordinates[0]]
        
        for coord in sorted_coordinates[1:]:
            if abs(coord[0] - current_group[-1][0]) < 50:
                current_group.append(coord)
            else:
                groups.append(current_group)
                current_group = [coord]
        
        # Append the last group
        groups.append(current_group)
        
        # Step 2: Sort each group by the second index (column values)
        for group in groups:
            group.sort(key=lambda x: x[1])
        
        # Step 3: Combine the groups back together
        sorted_coordinates = [coord for group in groups for coord in group]

    return sorted_coordinates

def fill_gaps(sorted_coordinates):
    addition=0
    for num in range(len(sorted_coordinates)-1):
        
        if abs(sorted_coordinates[num][0] - sorted_coordinates[num+1][0])< 50 :
            if sorted_coordinates[num+1][1] - sorted_coordinates[num][1] > 150:
                x=(sorted_coordinates[num+1][0] + sorted_coordinates[num][0])/2
                y=(sorted_coordinates[num+1][1] + sorted_coordinates[num][1])/2
                p1=sorted_coordinates[num+1][5]
                p2=sorted_coordinates[num+1][4]
                p3=sorted_coordinates[num][3]
                p4=sorted_coordinates[num][2]
                sorted_coordinates.insert(num+1,[x,y,p1,p2,p3,p4])
           
                addition+=1
                
    return sorted_coordinates


def squared_board(sorted_coordinates):
    square_num=1
    for cor in sorted_coordinates:
          cv2.putText(img = board_squared,text = str(square_num),
            org = (int(cor[1])-30, int(cor[0])),
            fontFace = cv2.FONT_HERSHEY_DUPLEX,fontScale = 1,color = (125, 246, 55),thickness = 3)
          square_num+=1

    return board_squared
      
    

<br><br>

## Function for writing extracted coordinates to a CSV file

In [3]:
def validate_board(sorted_coordinates_filled):

    
    if len(sorted_coordinates_filled)==64:
        for i in range(len(sorted_coordinates_filled)):
            if i not in [7,15,23,31,39,47,55,63]:
                if  sorted_coordinates_filled[i+1][0] - sorted_coordinates_filled[i][0] < 50 and abs(sorted_coordinates_filled[i+1][1] - sorted_coordinates_filled[i][1])<150 :
                    pass
                else: 
                    print(f"Problem {i}")
                    return False

        with open('dynamic-chess-coordinates.csv', mode='w', newline='') as file:
            writer = csv.writer(file)
            
            # columns
            writer.writerow(["ycenter","xcenter",'x1', 'y1', 'x2', 'y2', 'x3', 'y3', 'x4', 'y4']) 

            
        
        
            for coordinate in sorted_coordinates_filled:
                writer.writerow([coordinate[0],coordinate[1],coordinate[2][0], coordinate[2][1],
                                         coordinate[3][0], coordinate[3][1],
                                         coordinate[4][0], coordinate[4][1],
                                         coordinate[5][0], coordinate[5][1]])

            print(" Coordinates are ready to use ")
         

            
    else:
        return False
        


<br><br>

## Calibration of Camera 

#### Camera  : OAK-D Lite 

In [4]:
 # Create pipeline
pipeline = dai.Pipeline()

# Define sources and outputs
cam_rgb = pipeline.create(dai.node.ColorCamera)
xout_video = pipeline.create(dai.node.XLinkOut)
xout_video.setStreamName("video")

# Properties
cam_rgb.setBoardSocket(dai.CameraBoardSocket.RGB)
cam_rgb.setResolution(dai.ColorCameraProperties.SensorResolution.THE_1080_P)

# Linking
cam_rgb.video.link(xout_video.input)

# Connect to device and start pipeline
with dai.Device(pipeline) as device:
    # Start pipeline
    video_queue = device.getOutputQueue(name="video", maxSize=30, blocking=False)

    while True:
        # Get a frame
        video_frame = video_queue.get()
        frame = video_frame.getCvFrame()



        print(video_frame.getFrame().shape)

        
        gaussian_blur_image = gaussian_blur(frame)
        otsu_binary_image =  otsu_threshold(gaussian_blur_image)
        canny_edge_image =  canny_edge_detection(otsu_binary_image)
        dilation1_image = dilation1(canny_edge_image)
        board_contours = hough_lines(dilation1_image)
        board_squared , square_centers = find_contours(board_contours)
        
        sorted_coordinates = sort_coordinates(square_centers)
        sorted_coordinates_filled_gap = fill_gaps(sorted_coordinates)
        board_squared = squared_board(sorted_coordinates_filled_gap)

        validate_board(sorted_coordinates_filled_gap)

        # Display the processed frame
        cv2.imshow("Resized Frame", board_squared)

        time.sleep(1)

        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cv2.destroyAllWindows()

C:\Users\sirom\AppData\Local\Temp\ipykernel_8212\3103975502.py:10: DeprecationWarning: RGB is deprecated, use CAM_A or address camera by name instead.
  cam_rgb.setBoardSocket(dai.CameraBoardSocket.RGB)


(1620, 1920)
(1620, 1920)
(1620, 1920)
(1620, 1920)
(1620, 1920)
(1620, 1920)
(1620, 1920)
(1620, 1920)
(1620, 1920)
(1620, 1920)
(1620, 1920)
(1620, 1920)
(1620, 1920)
(1620, 1920)
(1620, 1920)
(1620, 1920)
(1620, 1920)
(1620, 1920)
(1620, 1920)
(1620, 1920)
(1620, 1920)
(1620, 1920)
(1620, 1920)
(1620, 1920)
(1620, 1920)
(1620, 1920)
(1620, 1920)
(1620, 1920)
(1620, 1920)
(1620, 1920)
(1620, 1920)
(1620, 1920)
(1620, 1920)
(1620, 1920)
(1620, 1920)
(1620, 1920)
(1620, 1920)
(1620, 1920)
(1620, 1920)
(1620, 1920)
(1620, 1920)
(1620, 1920)
(1620, 1920)
(1620, 1920)
(1620, 1920)
(1620, 1920)
(1620, 1920)
(1620, 1920)
(1620, 1920)
(1620, 1920)
(1620, 1920)
(1620, 1920)
(1620, 1920)
(1620, 1920)
(1620, 1920)
(1620, 1920)
(1620, 1920)
(1620, 1920)
(1620, 1920)
(1620, 1920)
(1620, 1920)
(1620, 1920)
(1620, 1920)
(1620, 1920)
(1620, 1920)
(1620, 1920)
(1620, 1920)
(1620, 1920)
(1620, 1920)
(1620, 1920)
(1620, 1920)
(1620, 1920)
(1620, 1920)
(1620, 1920)
(1620, 1920)
(1620, 1920)
(1620, 1920)

<br><br><br><br><br><br><br><br><br><br><br><br>